# Test constraints

**Context:** With my simple test setup created in `Develop_SurveyPlanner.ipynb` I noticed on 2023-11-14 that the field with ID 1 (in the database, i.e. 0 in a python list) is observable on JD 2460317.5-2460323.5 and 2460326.5-2460328.5, but not 2460324.5 and 2460325.5. This is odd. Is this a bug or is this correct? In this notebook I am looking into this potential issue.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from astropy.coordinates import AltAz, Angle, EarthLocation, get_sun, SkyCoord
from astropy.time import Time, TimeDelta
import astropy.units as u
import numpy as np

import constraints as c
from fieldgrid import FieldGrid, FieldGridIsoLat
from surveyplanner import Field, ObsWindow, Telescope, SurveyPlanner

## Create test grid

I copy the Northern test grid from `Develop_SurveyPlanner.ipynb`:

In [3]:
# north:
fov = np.radians(5.)
overlap_ns = fov / 2.
overlap_ew = np.radians(1.)
tilt = np.radians(0.)
gal_lat_lim = np.radians(30.)
dec_lim_south = -fov / 2.

fields_north = FieldGridIsoLat(
        fov, overlap_ns=overlap_ns, overlap_ew=overlap_ew, tilt=tilt, gal_lat_lim=gal_lat_lim, dec_lim_south=dec_lim_south)
print('\n', fields_north)

Create fields..
    Done                                                    
Final number of fields: 1056

 FieldGridIsoLat : Iso-latitudinal field grid
Field of view:     5.0000 deg
Overlap N-S        2.5000 deg
Overlap E-W        1.0000 deg
Tilt:             +0.0000 deg
Gal. lat. lim:    30.0000 deg
Dec. lim. N:      90.0000 deg
Dec. lim. S:      -2.5000 deg
Number of fields: 1056


In [4]:
field_id = 0
field_coord = SkyCoord(fields_north.center_ras[field_id], fields_north.center_decs[field_id], unit='rad')

## Create telescope

In [5]:
name = 'Skinakas'
lat = Angle('35:12:43 deg')
lon = Angle('24:53:57 deg')
height = 1750.
utc_offset = 2.
telescope = Telescope(lat, lon, height, utc_offset, name=name)

Telescope Skinakas created.


## Set constraints

I use the constraints for the Northern survey:

In [6]:
twilight = 'nautical'
airmass_limit = c.AirmassLimit(2.)
hourangle_limit = c.HourangleLimit(5.33)
moon_distance = c.MoonDistance(10.)
constraints = c.Constraints()
constraints.add(airmass_limit)
constraints.add(hourangle_limit)
constraints.add(moon_distance)

## Find observability windows

To identify blocks I use this function that I copied from `surveyplanner.Field._true_blocks()`:

In [7]:
def true_blocks(observable):
    """Find blocks of successive True's.

    Parameters
    ----------
    observable : nump.ndarray
        Boolean-type 1dim-array.

    Returns
    -------
    list
        Each element corresponds to one block of True's. The element is a
        list of two integers, the first marking the first index of the
        block, the second marking the last True entry of the block.
    """

    i = 0
    periods = []

    # iterate through array:
    while i < observable.size-1:
        if ~np.any(observable[i:]):
            break
        j = np.argmax(observable[i:]) + i
        k = np.argmax(~observable[j:]) + j
        if j == k and j != observable.size-1:
            k = observable.size
        periods.append((j,k-1))
        i = k

    return periods

Set time range:

In [9]:
date_start = Time('2024-01-01')
date_stop = Time('2024-01-20')
twilight = 'nautical'

for date in np.arange(date_start.jd, date_stop.jd):
    dt = Time(date, format='jd').datetime
    time_sunset, time_sunrise = telescope.get_sun_set_rise(dt.year, dt.month, dt.day, twilight)
    frame = telescope.get_frame(time_sunset, time_sunrise, 10*u.min)
    observable = constraints.get(field_coord, frame)
    blocks = true_blocks(observable)
    observable_airmass = airmass_limit.get(field_coord, frame)
    observable_hourangle = hourangle_limit.get(field_coord, frame)
    observable_moon = moon_distance.get(field_coord, frame)
    print(date, time_sunset, time_sunrise)
    print(np.sum(observable), np.sum(observable_airmass), np.sum(observable_hourangle), np.sum(observable_moon))
    
    for block_id0, block_id1 in blocks:
        print(frame.obstime[block_id0], frame.obstime[block_id1])
    
    if not len(blocks):
        print('Field not observable.')
        
    print()

2460310.5 2024-01-01T16:18:28.689 2024-01-02T04:29:14.721
17 17 28 74
2024-01-01T16:18:28.689 2024-01-01T18:58:28.689

2460311.5 2024-01-02T16:19:11.498 2024-01-03T04:29:27.279
17 17 28 74
2024-01-02T16:19:11.498 2024-01-02T18:59:11.498

2460312.5 2024-01-03T16:19:55.302 2024-01-04T04:29:38.205
16 16 27 73
2024-01-03T16:19:55.302 2024-01-03T18:49:55.302

2460313.5 2024-01-04T16:20:40.061 2024-01-05T04:29:47.482
16 16 27 73
2024-01-04T16:20:40.061 2024-01-04T18:50:40.061

2460314.5 2024-01-05T16:21:25.739 2024-01-06T04:29:55.093
16 16 26 73
2024-01-05T16:21:25.739 2024-01-05T18:51:25.739

2460315.5 2024-01-06T16:22:12.294 2024-01-07T04:30:01.026
15 15 26 73
2024-01-06T16:22:12.294 2024-01-06T18:42:12.294

2460316.5 2024-01-07T16:22:59.688 2024-01-08T04:30:05.276
15 15 25 73
2024-01-07T16:22:59.688 2024-01-07T18:42:59.688

2460317.5 2024-01-08T16:23:47.877 2024-01-09T04:30:07.806
14 14 25 73
2024-01-08T16:23:47.877 2024-01-08T18:33:47.877

2460318.5 2024-01-09T16:24:36.649 2024-01-10T04:

## Results

The field is not observable on JD 2460324 and 2460325 because it is too close to the Moon. I cross-checked this result with [staralt](http://catserver.ing.iac.es/staralt/). It is correct. The days before the Moon is getting closer to the field, the days after it is moving away from the field. Those two days the Moon is very close to the field.

**Conclusion:** Everything is working fine.